In [1]:
#!pip install tensorflow==2.17.1
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import numpy as np
import tensorflow as tf
import os
import shutil

import matplotlib.pyplot as plt

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


2024-12-02 15:39:07.261097: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 15:39:07.278992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 15:39:07.298812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 15:39:07.304870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 15:39:07.319989: I tensorflow/core/platform/cpu_feature_guar

In [3]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
tf.__version__

'2.17.1'

In [5]:
#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
#!unzip data.zip

In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

I0000 00:00:1733153949.620242   20128 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733153949.668310   20128 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733153949.668539   20128 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733153949.669350   20128 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [7]:
model.compile(loss='binary_crossentropy',\
             optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),\
             metrics=['acc'])

In [8]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

In [10]:
train_ds = train_gen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.


In [11]:
validation_ds = val_gen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 201 images belonging to 2 classes.


In [12]:
data_batch, labels_batch = next(iter(train_ds))
print('data batch shape:', data_batch.shape)
print('labels batch shape:', labels_batch.shape)

data batch shape: (20, 200, 200, 3)
labels batch shape: (20,)


In [13]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=validation_ds
)

Epoch 1/10


I0000 00:00:1733153951.695844   20177 service.cc:146] XLA service 0x7f0588003610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733153951.695880   20177 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-02 15:39:11.716853: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-02 15:39:11.813301: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-12-02 15:39:12.235414: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 464.45MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 3/40 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - acc: 0.4917 - loss: 0.6983

I0000 00:00:1733153953.066849   20177 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - acc: 0.5146 - loss: 0.7030

2024-12-02 15:39:17.579490: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.53MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-12-02 15:39:17.625882: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 338.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 155ms/step - acc: 0.5157 - loss: 0.7026 - val_acc: 0.5323 - val_loss: 0.6834
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.5393 - loss: 0.6683 - val_acc: 0.6269 - val_loss: 0.6395
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.7069 - loss: 0.5932 - val_acc: 0.6020 - val_loss: 0.6381
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - acc: 0.7054 - loss: 0.5711 - val_acc: 0.6468 - val_loss: 0.6410
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.7130 - loss: 0.5416 - val_acc: 0.6567 - val_loss: 0.6132
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - acc: 0.7407 - loss: 0.5254 - val_acc: 0.6020 - val_loss: 0.6574
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.7155 - loss: 0.5242 - val_acc: 0.6567 - val_loss: 0.6033
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - acc: 0.7529 - loss: 0.4956 - val_acc: 0.6866 - val_loss: 0.5958
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step - acc: 0.7589 -

In [14]:
acc = history.history['acc']
f"median of training accuracy: {np.median(acc):.3f}"

'median of training accuracy: 0.704'

In [15]:
loss = history.history['loss']
f"standard deviation training loss: {np.std(loss):.3f}"

'standard deviation training loss: 0.064'

In [16]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [17]:
test_gen = ImageDataGenerator(rescale=1./255)

In [18]:
train_ds = train_gen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.


In [19]:
validation_ds = test_gen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 201 images belonging to 2 classes.


In [20]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=validation_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 269ms/step - acc: 0.6578 - loss: 0.5989 - val_acc: 0.7015 - val_loss: 0.5731
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 265ms/step - acc: 0.6899 - loss: 0.5830 - val_acc: 0.7065 - val_loss: 0.5741
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - acc: 0.7030 - loss: 0.5801 - val_acc: 0.7114 - val_loss: 0.5850
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 260ms/step - acc: 0.7005 - loss: 0.5558 - val_acc: 0.7164 - val_loss: 0.5613
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 261ms/step - acc: 0.6969 - loss: 0.5649 - val_acc: 0.7164 - val_loss: 0.5741
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - acc: 0.7351 - loss: 0.5625 - val_acc: 0.7264 - val_loss: 0.5510
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 272ms/step - acc: 0.7032 - loss: 0.5906 - val_acc: 0.6915 - val_loss: 0.6292
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - acc: 0.7330 - loss: 0.5552 - val_acc: 0.6766 - val_loss: 0.6086
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 266ms/

In [21]:
val_loss_aug = history.history['val_loss']
f"Mean of test loss: {np.mean(val_loss_aug):.2f}"

'Mean of test loss: 0.58'

In [22]:
val_acc_aug = history.history['val_acc']
f"Average of test accuracy: {np.mean(val_acc_aug[5:10]):.2f}"

'Average of test accuracy: 0.70'